In [1]:
import pandas as pd

fp = '../2D_5coeffs.csv'
# fp = '../2D-besthalf.csv'
df = pd.read_csv(fp)
p1 = 'P1'
p2 = 'P2'
p3 = 'P3'
p4 = 'P4'
p5 = 'P5'
target = df.columns.to_list()[-1]
features = df.columns.to_list()[:-1]
input_cols = features
df

,P1,P2,P3,P4,P5,Cost
0,-45,107,-45,102,-105,12286.0
1,-45,107,-45,101,-105,12290.0
2,-45,107,-45,100,-105,12290.0
3,-50,102,-49,90,-94,12292.0
4,-43,98,-41,98,-105,12292.0
...,...,...,...,...,...,...
36172,87,-100,56,-87,65,45078.0
36173,66,-98,51,-63,77,45100.0
36174,66,-96,51,-63,77,45110.0
36175,95,-99,37,-71,89,45122.0


In [2]:
df.describe()

,P1,P2,P3,P4,P5,Cost
count,36177.000000,36177.000000,36177.000000,36177.000000,36177.000000,36177.000000
mean,-38.295851,65.586699,-39.533765,65.111590,-67.706941,14428.408713
std,33.097651,49.738359,29.165907,38.755047,46.006320,6573.087008
min,-104.000000,-100.000000,-105.000000,-100.000000,-117.000000,12286.000000
25%,-54.000000,46.000000,-49.000000,57.000000,-97.000000,12304.000000
50%,-44.000000,94.000000,-42.000000,73.000000,-84.000000,12362.000000
75%,-40.000000,98.000000,-38.000000,92.000000,-63.000000,12840.000000
max,102.000000,111.000000,100.000000,110.000000,102.000000,54556.000000


In [3]:
df.keys()

Index(['P1', 'P2', 'P3', 'P4', 'P5', 'Cost'], dtype='object')

In [14]:
# df.plot(x=p3, y=p4, kind='scatter', c=target)

In [15]:
# df.hist(column=target,bins=100)

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create the pairplot
# g = sns.pairplot(df, hue=target, vars=features, corner=True, palette='viridis',
#                 height=5)

# # Set y-axes limits for all plots
# for i in range(g.axes.shape[0]):
#     for j in range(g.axes.shape[1]):
#         if j == 0:
#             g.axes[i][j].set_ylim(-120, 120)
#         if i == g.axes.shape[0]-1:
#             g.axes[i][j].set_xlim(-120, 120)

# # Display the plot
# plt.show()

In [7]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold

from sklearn.base import clone
from collections import defaultdict

def train_cv(model, data, input_cols, n_splits=5, model_log=False):
    """
    Use cross validation.
    Train models with n different splits to avg test eval
    """
    cv = KFold(n_splits=n_splits, random_state=seed, shuffle=True)
    for (trn_ind, test_ind) in cv.split(data):
        trn = data.iloc[trn_ind]
        test = data.iloc[test_ind] 
        pipe = train(model, X_trn=trn[input_cols], y_trn=trn[target],
                    model_log=model_log)
        yield pipe, evaluate(pipe, X_tst=test[input_cols], y_tst=test[target])

def train(model, X_trn, y_trn, model_log=False):
    # using a transformer on the outputs (y) to get log VR, this allows
    # us to model in log space, but it is automatically reverted for metrics
    if model_log == True:
        regr = TransformedTargetRegressor(regressor=clone(model),
                                          transformer=LogTransformer)
    else:
        regr = clone(model)
    
    p = Pipeline(steps=[
        ('scale', StandardScaler()), 
        ('model', regr)])
    p.fit(X_trn, y_trn)
    return p # return fitted pipeline

def evaluate(p: Pipeline, X_tst, y_tst):
    """return metrics & pipeline (r2, rmse, mae, pipe)"""
    pred = p.predict(X_tst)
    return (r2_score(y_tst, pred), 
            mean_squared_error(y_tst, pred)**0.5, 
            mean_absolute_error(y_tst, pred))

seed = 123

model = MLPRegressor(random_state=seed, hidden_layer_sizes=[30,30], solver='lbfgs', alpha=0.0001, max_iter=2500)

In [8]:
# import numpy as np
# from sklearn.model_selection import ValidationCurveDisplay

# _ = ValidationCurveDisplay.from_estimator(
#     model,
#     df[input_cols],
#     df[target],
#     param_name="alpha",
#     param_range=np.geomspace(1e-7, 1e-3, num=5),
#     score_type="both"
# )

In [9]:
# # Cross-validate the model settings:
# resp = train_cv(model, df, input_cols)

# results = defaultdict(list)

# for pipe, (r2, rmse, mae) in resp:
#     results["r²"].append(round(r2, 3))
#     results["rmse"].append(round(rmse, 3))
#     results["mae"].append(round(mae, 3))

# results

In [10]:
trained_pipe = train(model, df[input_cols], df[target])

/home/mik/aston/phd/junaidrasheed/LPPaver/bisection_optimisation_2D/models/opt_venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [11]:
trained_pipe.predict(pd.DataFrame([[0,100,-100,100,-100]], columns=input_cols))

array([12938.34360957])

In [12]:
from joblib import dump, load

dump(trained_pipe, 'lppaver_2d_bisect_5_coeffs.joblib') 

['lppaver_2d_bisect_5_coeffs.joblib']

In [13]:
loaded_pipe = load('lppaver_2d_bisect_5_coeffs.joblib')
loaded_pipe.predict(pd.DataFrame([[0,100,-100,100,-100]], columns=input_cols))

array([12938.34360957])